## With Diversification constraint

In [3]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.optimize import root_scalar
from sklearn.covariance import LedoitWolf

In [4]:
df = pd.read_csv('_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df = df.loc['2022-01-01':'2024-01-01']
returns = df.pct_change().dropna()

In [5]:
#parameters
r=0.05  # risk-free rate
portfolio_value = 1000000  # initial portfolio value
n=len(df.columns)  # number of assets
volatility_target = 0.25  # target annualized volatility
x_max=0.07  # maximum weight per asset
TRADING_DAYS = 365 # number of trading days in a year

# backtest parameters
rebalance_frequency = 30 # monthly rebalancing
window_size = 90  # 3 months rolling window

D_min=0.70 #min of diversification

In [6]:
#fees parameters
c_fee = 0.001  # 0.1% per trade (buy or sell)

total_fees = 0.0
fees_list = []          
fees_dates = []        


In [7]:
#compute rolling statistics
rolling_mean = returns.rolling(window=window_size).mean().shift(1) 
#rolling_cov = returns.rolling(window=window_size).cov().shift(1) # not used anymore with LedoitWolf


In [8]:


def optimize_target_vol(mu_daily, Sigma_daily, r_annual=r, target_vol_annual=volatility_target, w_max=x_max):
    mu_daily = np.asarray(mu_daily).reshape(-1) 
    n = mu_daily.shape[0]
    Sigma_daily = np.asarray(Sigma_daily) 

    # Consistent annualization 
    Sigma_ann = TRADING_DAYS * Sigma_daily
    mu_ann = TRADING_DAYS * mu_daily

   
    Sigma_ann = Sigma_ann + 1e-10 * np.eye(n)

    w = cp.Variable(n)  # risky weights
    w_cash = 1 - cp.sum(w) # cash weight
    s=cp.sum(w)
    
    # objective: expected annual return risky + cash
    objective = cp.Maximize(mu_ann @ w + r_annual * w_cash)

    constraints = [
        w >= 0,
        w <= w_max,
        cp.sum(w) <= 1,
        cp.norm(w,2)<=(1/np.sqrt(n*D_min))*s, # minimum diversification constraint
        cp.quad_form(w, Sigma_ann) <= target_vol_annual**2
    ]

    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS, verbose=False)

    if w.value is None:
        raise RuntimeError("Optimization failed (infeasible/solver).")

    w_risky = np.array(w.value).reshape(-1)
    w0 = float(1 - w_risky.sum())
    return w0, w_risky


In [9]:
# rebalance dates (every 30 days starting from window_size)
rebalance_dates = returns.index[window_size::rebalance_frequency]

portfolio_values = []
weights_history = []

V = portfolio_value

# initialize weights (before first optimization)
w0 = 1.0
w_risky = np.zeros(n)

for t in returns.index[window_size:]:  # start when we have lookback data
    # 1) if it's a rebalance date, calculate new weights
    if t in rebalance_dates:
        mu = rolling_mean.loc[t].values
        
        #old Sigma
        """
        Sigma = rolling_cov.loc[t].values  # must be (n,n)

        # Cleaning + symmetrization
        Sigma = np.asarray(Sigma, dtype=float)
        Sigma = np.nan_to_num(Sigma, nan=0.0, posinf=0.0, neginf=0.0)
        Sigma = 0.5 * (Sigma + Sigma.T)  # <-- symmetrization
        # Shift diagonal if min eigenvalue < 0
        eig_min = np.linalg.eigvalsh(Sigma).min()
        if eig_min < 0:
            Sigma = Sigma + (-eig_min + 1e-10) * np.eye(Sigma.shape[0])
        """
        # COVARIANCE SHRINKAGE (Ledoit–Wolf) 
        # Returns window that stops at t-1 (no lookahead)
        idx = returns.index.get_loc(t)
        window_ret = returns.iloc[idx-window_size:idx]  

        # LedoitWolf doesn't accept NaN/inf 
        X = window_ret.replace([np.inf, -np.inf], np.nan)
        X = X.apply(lambda col: col.fillna(col.mean()), axis=0).fillna(0.0)

        lw = LedoitWolf().fit(X.values)
        Sigma = lw.covariance_  # shrunk covariance (much more stable)
        Sigma = 0.5 * (Sigma + Sigma.T)  # numerical safety
        
        w0_target, w_risky_target = optimize_target_vol(
            mu, Sigma,
            r_annual=r,
            target_vol_annual=volatility_target,
            w_max=x_max
        )
        
        V_before = V
        s_old = V_before * w_risky              # risky dollars before
        s_new = V_before * w_risky_target       # target risky dollars after
        traded_notional = np.sum(np.abs(s_new - s_old))
        fee_t = c_fee * traded_notional

        total_fees += fee_t
        fees_list.append(fee_t)
        fees_dates.append(t)

   
        V = V_before - fee_t

        # 2) Effective weights after paying fees
        w_risky = s_new / V
        w0 = 1.0 - w_risky.sum()

        # numerical safety (just in case)
        w_risky = np.clip(w_risky, 0.0, 1.0)
        w0 = max(0.0, 1.0 - w_risky.sum())
        weights_history.append((t, w0, w_risky.copy()))

    # 3) apply the daily return with current weights 
    daily_rf = r / TRADING_DAYS
    port_ret = w0 * daily_rf + float(np.dot(w_risky, returns.loc[t].values))
    V = V * (1 + port_ret)

    portfolio_values.append((t, V))

portfolio_values = pd.Series(
    [v for _, v in portfolio_values],
    index=[d for d, _ in portfolio_values],
    name="V"
)


In [10]:
#computation of the performance metrics
def annualized_return(portfolio_values,initial_value):
    T=len(portfolio_values.index)-1
    return TRADING_DAYS/T*(portfolio_values.iloc[-1]- initial_value)/initial_value

print("Annualized Return:", annualized_return(portfolio_values, portfolio_value))

Annualized Return: 0.16934550455851546


In [11]:
def portfolio_daily_returns(portfolio_values):
    return portfolio_values.pct_change().dropna()

In [12]:
def annualized_volatility(portfolio_values):
    r = portfolio_daily_returns(portfolio_values)
    return np.sqrt(TRADING_DAYS) * r.std(ddof=1) 

print("Annualized Volatility:", annualized_volatility(portfolio_values))

Annualized Volatility: 0.22709222155863873


In [13]:
def sharpe_ratio(portfolio_values, r_annual=0.05):
    r = portfolio_daily_returns(portfolio_values)
    r_daily_rf = r_annual / TRADING_DAYS
    excess_mean = (r - r_daily_rf).mean()
    vol_daily = r.std(ddof=1)
    return np.sqrt(TRADING_DAYS) * excess_mean / vol_daily 

print("Sharpe Ratio:", sharpe_ratio(portfolio_values, r_annual=r))

Sharpe Ratio: 0.507567787069358


In [14]:
def max_drawdown(portfolio_values):
    V = portfolio_values.values
    running_max = np.maximum.accumulate(V)
    drawdowns = 1.0 - (V / running_max)
    return np.max(drawdowns)

print("Maximum Drawdown:", max_drawdown(portfolio_values))

Maximum Drawdown: 0.235270404810774


In [15]:
def diversification_D(w_risky):
    w = np.array(w_risky, dtype=float)
    s = w.sum()
    H = (w @ w) / (s * s)
    n = len(w)
    return (1.0 / n) * (1.0 / H)

In [16]:
def average_diversification(weights_history):
    Ds = []
    for w_full in weights_history:
        w_full = np.asarray(w_full).reshape(-1)
        w_risky = w_full[1:]  # exclude cash
        Ds.append(diversification_D(w_risky))
    return float(np.mean(Ds)) if len(Ds) > 0 else np.nan

print("Average Diversification D:", average_diversification([np.hstack(([w0], w_risky)) for _, w0, w_risky in weights_history]))

Average Diversification D: 0.6031139478919428


In [17]:
# Asset allocation analysis
print("="*80)
print("ASSET ALLOCATION ANALYSIS")
print("="*80)

# Get asset names
asset_names = df.columns.tolist()

# Create a matrix of all weights over time
weights_matrix = np.array([w_risky for _, _, w_risky in weights_history])

# Calculate statistics for each asset
print(f"\n{'Asset':<20} {'Average':<12} {'Min':<12} {'Max':<12} {'Count >0':<12}")
print("-"*80)

for i, asset_name in enumerate(asset_names):
    weights_asset = weights_matrix[:, i]
    mean_weight = weights_asset.mean()
    min_weight = weights_asset.min()
    max_weight = weights_asset.max()
    nb_nonzero = np.sum(weights_asset > 0.001)  # number of times weight > 0.1%
    
    print(f"{asset_name:<20} {mean_weight:>10.2%}  {min_weight:>10.2%}  {max_weight:>10.2%}  {nb_nonzero:>10}")

print("\n" + "="*80)
print(f"Average cash allocation: {np.mean([w0 for _, w0, _ in weights_history]):.2%}")
print(f"Average risky assets allocation: {(1 - np.mean([w0 for _, w0, _ in weights_history])):.2%}")

ASSET ALLOCATION ANALYSIS

Asset                Average      Min          Max          Count >0    
--------------------------------------------------------------------------------
bitcoin                   1.97%       0.00%       4.17%          17
ethereum                  1.70%       0.00%       3.96%          17
solana                    1.81%       0.00%       5.01%          16
polkadot                  0.99%       0.00%       2.22%          17
avalanche-2               1.23%       0.00%       6.26%          17
tron                      2.07%       0.00%       4.98%          17
thorchain                 1.97%       0.00%       7.00%          16
ripple                    2.00%       0.00%       6.86%          17
vechain                   1.32%       0.00%       3.26%          17
matic-network             1.07%       0.00%       3.06%          15
chainlink                 1.89%       0.00%       5.81%          17
the-graph                 1.21%       0.00%       3.14%          17
arw

In [18]:
print("\n" + "="*100)
print(" "*35 + "PERFORMANCE SUMMARY")
print("="*100)

# Calculation of all metrics
ann_return = annualized_return(portfolio_values, portfolio_value)
ann_vol = annualized_volatility(portfolio_values)
sharpe = sharpe_ratio(portfolio_values, r_annual=r)
mdd = max_drawdown(portfolio_values)
avg_div = average_diversification([np.hstack(([w0], w_risky)) for _, w0, w_risky in weights_history])
avg_cash = np.mean([w0 for _, w0, _ in weights_history])
avg_risky = 1 - avg_cash
fee_cost = total_fees / portfolio_value


# Formatted display
print(f"\n{'Metric':<40} {'Value':>20}")
print("-"*100)
print(f"{'Annualized return':<40} {ann_return:>19.2%}")
print(f"{'Annualized volatility':<40} {ann_vol:>19.2%}")
print(f"{'Sharpe ratio':<40} {sharpe:>20.4f}")
print(f"{'Maximum drawdown':<40} {mdd:>19.2%}")
print(f"{'Average diversification (D)':<40} {avg_div:>20.4f}")
print(f"{"Total Fees ($)":<40} {total_fees:>20.4f}")
print(f"{"Fee cost":<40} {fee_cost:>19.2%}")

print("-"*100)
print(f"{'Average cash allocation':<40} {avg_cash:>19.2%}")
print(f"{'Average risky assets allocation':<40} {avg_risky:>19.2%}")
print("-"*100)
print(f"{'Initial portfolio value':<40} {portfolio_value:>20,.0f}")
print(f"{'Final portfolio value':<40} {portfolio_values.iloc[-1]:>20,.2f}")
print(f"{'Total gain':<40} {(portfolio_values.iloc[-1] - portfolio_value):>20,.2f}")
print("="*100 + "\n")


                                   PERFORMANCE SUMMARY

Metric                                                  Value
----------------------------------------------------------------------------------------------------
Annualized return                                     16.93%
Annualized volatility                                 22.71%
Sharpe ratio                                           0.5076
Maximum drawdown                                      23.53%
Average diversification (D)                            0.6031
Total Fees ($)                                      3661.2229
Fee cost                                               0.37%
----------------------------------------------------------------------------------------------------
Average cash allocation                               69.32%
Average risky assets allocation                       30.68%
----------------------------------------------------------------------------------------------------
Initial portfolio value   